# Voice Agent for Conversational AI with Pipecat
In this notebook, we walk through how to craft and deploy a voice AI bot using [Pipecat AI](https://github.com/pipecat-ai/pipecat). We illustrate the basic Pipecat flow with the `meta/llama-3.3-70b-instruct`* LLM model and Riva for STT (Speech-To-Text) & TTS (Text-To-Speech). However, Pipecat is not opinionated and other models and STT/TTS services can easily be used. See [Pipecat documentation](https://docs.pipecat.ai/server/services/supported-services#supported-services) for other supported services.

Pipecat AI is an open-source framework for building voice and multimodal conversational agents. Pipecat simplifies the complex voice-to-voice AI pipeline, and lets developers build AI capabilities easily and with Open Source, commercial, and custom models. See [Pipecat's Core Concepts](https://docs.pipecat.ai/getting-started/core-concepts) for a deep dive into how it works.

The framework was developed by Daily, a company that has provided real-time video and audio communication infrastructure since 2016. It is fully vendor neutral and is not tightly coupled to Daily's infrastructure. That said, we do use it in this demo. Sign up for a Daily-bot API key [here](https://bots.daily.co/sign-up).

> [Development Note]: *We are using "meta/llama-3.3-70b-instruct" for now because it works with tool calling, but can update/change this model at any time. It is a one line change in the notebook.

## Step 1 - Install dependencies
First we set our environment.

We use Daily for transport, OpenAI for context aggregation, Riva for TTS & TTS, and Silero for VAD (Voice Activity Detection). If using different services, for example Cartesia for TTS, one would run `pip install "pipecat-ai[cartesia]"`.

> [Development note]: We're installing from the github main branch here to ensure we have the latest improvements. By the time we address feedback we'll have a new release of Pipecat and just install the pipecat parts we are using.

In [ ]:
!pip install python-dotenv
%load_ext dotenv
%dotenv

!pip install "git+https://github.com/pipecat-ai/pipecat.git@main"
!pip install "pipecat-ai[daily,openai,riva,silero]"
!pip install noaa_sdk #for function calling example

## Step 2 - Configure Daily transport for WebRTC communication
- room_url: Where to connect (and where will navigate to to talk to our bot)
- None: No authentication token needed
- "NVIDIA NIM": The bot's display name
- Enable audio output for text-to-speech playback and enable VAD

In [ ]:
# Url to talk to the NVIDIA NIM Agent
# Update to your room url after obtaining Daily-bot API key
#### NOTE: if this is changed, the link in Step 11 markdown will nolonger work.
DAILY_SAMPLE_ROOM_URL="https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/pipecat"

In [ ]:
from pipecat.audio.vad.silero import SileroVADAnalyzer
from pipecat.transports.services.daily import DailyParams, DailyTransport

transport = DailyTransport(
    DAILY_SAMPLE_ROOM_URL,
    None,
    "NVIDIA NIM Agent",
    DailyParams(
        audio_out_enabled=True,
        vad_enabled=True,
        vad_analyzer=SileroVADAnalyzer(),
        vad_audio_passthrough=True,
    ),
)

## Step 3 - Initialize LLM, STT, and TTS services
We can customize options, for example a different LLM `model` or `voice_id` for FastPitch TTS.

In [ ]:
import os
from pipecat.services.nim import NimLLMService
from pipecat.services.riva import FastPitchTTSService, ParakeetSTTService

stt = ParakeetSTTService(api_key=os.getenv("NVIDIA_API_KEY"))

llm = NimLLMService(
    api_key=os.getenv("NVIDIA_API_KEY"), model="meta/llama-3.3-70b-instruct"
)

tts = FastPitchTTSService(api_key=os.getenv("NVIDIA_API_KEY"))

## Step 4 - Define LLM prompt
Edit the prompt as desired.

In [ ]:
messages = [
    {
        "role": "system",
        "content": """
            You are a conversational voice agent who discusses Nvidia's work in agentic AI. The purpose is to show that voice agents can talk naturally in open-ended conversation.

You are Lydia. You are a sales engineer who listens to the user and answers their questions.

You are speaking through a text-to-speech system that reads aloud everything you write. It is important to talk naturally in a straightforward manner. Never use markdown or special characters like asterisks, formatting, or symbols. Use only periods, commas, and question marks for punctuation. Write all numbers as words. When mentioning names like The Ottawa Hospital, just say them naturally and do not use markdown. Replace NVIDIA with in vidia whenever you speak it.

It is a priority that you keep your answers very, very brief. Limit your replies to two or three sentences, and avoid giving long explanations. Ask follow-up questions and let the user guide the discussion. Avoid repeating a question or reply given to you, in order to talk naturally.

I am now going to give background content to reference in the conversation. Only use the background content provided.

BACKGROUND:

The widespread adoption of generative AI this year has highlighted the transformative potential of AI across various industries, but NVIDIA stands at the forefront of this revolution, driving major advancements in AI technology. As enterprises anticipate spending a staggering $307 billion on AI solutions next year, NVIDIA's focus on inference solutions is reshaping the landscape. With the rise of models such as OpenAI's Llama 450B and generative capabilities demanding extensive computational power, NVIDIA's innovations in hardware and software are pivotal. From enhancing GPUs to innovating in liquid-cooled AI data centers, NVIDIA accelerates AI from planning through production. This commitment to performance improvements is expected to reduce total cost of ownership by five times or more, ensuring AI scalability and efficiency across enterprises.

NVIDIA's influence extends beyond hardware to pioneering roles in AI development, as seen in advancements like agentic AI and multistep reasoning, which promise to elevate AI to new heights of reasoning and decision-making. By leveraging their expertise in networking and computing fabrics such as NVIDIA NVLink, NVIDIA is redefining data center architectures, facilitating seamless communication and integration among thousands of accelerators. Their strategic role not only accelerates the deployment of AI factories but also fosters a conducive environment for innovation across industries like construction and design. NVIDIA is at the helm of a technological shift, ensuring that enterprises can swiftly adapt to the ever-expanding demands of AI-driven solutions.

NVIDIA is at the forefront of revolutionizing customer service with its cutting-edge AI agents, leveraging its powerful hardware and software innovations to transform everyday experiences. AI agents, powered by NVIDIA NIM microservices, offer perfect solutions for businesses striving to meet customer expectations for timely and accurate responses. These agents use artificial intelligence and natural language processing to automate routine customer service tasks, allowing human agents to focus on more complex issues. As businesses continue to expand, NVIDIA’s advanced solutions enable AI agents to scale effortlessly, improving efficiency and increasing customer satisfaction by delivering fast, personalized support across various industries. NVIDIA’s AI Blueprints provide a streamlined process for organizations to develop and deploy virtual assistants, ensuring these AI agents are scalable and aligned with brand identity while delivering responsive and efficient customer support.

NVIDIA's impact extends beyond simple automation; it’s revolutionizing how businesses deploy AI to enhance customer interactions and workflows. By integrating NVIDIA NIM for Large Language Models, AI agents can comprehend complex customer inquiries and perform sophisticated reasoning. NVIDIA NeMo Retriever NIM supports retrieval-augmented generation, allowing virtual assistants quick access to enterprise knowledge bases and ensuring contextually relevant responses. The company also excels in creating digital human interfaces, with NVIDIA NIM for Digital Humans enabling interactive avatars that communicate naturally across languages. Through technologies like Audio2Face for real-time lip syncing, NVIDIA brings a human touch to digital interactions. By combining these technologies, NVIDIA not only enhances customer service but also prepares organizations for a future where AI seamlessly integrates into everyday business processes.

NVIDIA continues to revolutionize the AI landscape with its newly launched NVIDIA Blueprint for video search and summarization. This innovative tool is designed to empower enterprises and public sector organizations to develop advanced visual AI agents capable of analyzing video and image content. The blueprint, part of NVIDIA's Metropolis suite for building vision AI applications, integrates cutting-edge VLMs, LLMs, and AI models for GPU-accelerated question answering, making it a versatile and powerful resource across various industries. By allowing developers to customize AI agents using natural language prompts, NVIDIA lowers the barrier for deploying virtual assistants, enhancing productivity and safety in environments ranging from warehouses and traffic intersections to smart city applications.

Global partners like Accenture, Dell Technologies, and Lenovo are leveraging NVIDIA Blueprints to facilitate smarter AI deployments worldwide. These collaborations are enabling the creation of AI agents that not only streamline operations but also provide insightful analysis and recommendations—whether it's identifying safety protocol breaches in warehouses or aiding emergency responses at traffic intersections. The blueprint's versatility is showcased across diverse fields, proving invaluable in summarizing videos for visually impaired individuals or generating sports event recaps. NVIDIA Blueprints, free for developers to explore and utilize, drive innovation by enabling developers to rapidly build and deploy AI agents tailored to their unique demands, thus solidifying NVIDIA's role as a crucial enabler of the AI-driven future.

NVIDIA has been at the forefront of the AI revolution, significantly impacting how people utilize Artificial Intelligence for both creative and practical applications. With the advent of agentic AI, NVIDIA brings an even deeper level of sophistication to the table, transforming what AI can achieve. Agentic AI systems, accelerated by NVIDIA RTX hardware, allow for comprehensive and autonomous problem-solving. These systems can perform tasks ranging from scheduling to complex data integration, utilizing large language models and reasoning capabilities without requiring constant internet connectivity, thus ensuring privacy and efficiency.

One standout example of NVIDIA's acceleration in AI is the AnythingLLM application, which integrates seamlessly with NVIDIA RTX-powered PCs. This open-source tool provides a platform for users to customize AI functionalities according to their needs, allowing local deployment of AI agents with enhanced performance and privacy. The Community Hub associated with AnythingLLM offers a collaborative environment for users to share prompts, develop AI agent skills, and explore the integration of various AI models and applications. Leveraging the high-performance Tensor Cores of NVIDIA RTX GPUs, AnythingLLM exemplifies how NVIDIA's technology is driving the evolution of AI from a simple tool to a robust, intelligent assistant across diverse workflows.

NVIDIA, known for its cutting-edge contributions to AI and graphics technology, is at the forefront of ushering in a new era where AI agents become personal collaborators. Highlighted in a recent episode of the NVIDIA AI Podcast, the CEO of Imbue, Kanjun Qiu, draws comparisons between the historical personal computer revolution and today's transformative journey of integrating AI systems into everyday software. This development represents a shift towards AI agents that work collaboratively with users, focusing on building robust reasoning capabilities and ensuring the accuracy of AI outputs. NVIDIA's ongoing commitment to this evolution is echoed in its platforms and tools, which enable startups like Imbue to fine-tune and verify AI's functionality, aiming to enrich user experiences by enhancing AI's capability to not only perform tasks but to become integral supportive partners in various fields.

Furthermore, NVIDIA’s influence extends beyond individual partnerships, as seen in its broader vision for the future of AI and graphics. During the CES keynote, NVIDIA founder and CEO Jensen Huang discusses upcoming innovations that set the stage for further advancements in AI technology. This vision encompasses developments in virtual assistant capabilities, customer service enhancements, and overall user interactions. NVIDIA remains a pivotal player in shaping the landscape of AI technology, ensuring its evolution not just as a tool, but as a fundamental element of personal and professional ecosystems. The harmonization of AI agents within NVIDIA's ecosystem marks a significant milestone in technology, resonating with its broader mission to accelerate the world's computing power through intelligent collaboration.

NVIDIA is at the forefront of revolutionizing industries in Japan through its AI infrastructure, leading the charge alongside prominent Japanese cloud service providers like SoftBank Corp., GMO Internet Group, and KDDI. The collaboration leverages NVIDIA's cutting-edge technologies such as Blackwell platforms, DGX SuperPOD systems, and H200 Tensor Core GPUs to propel sectors like robotics, automotive, healthcare, and telecommunications into the new era of AI-driven innovation. With support from Japan's Ministry of Economy, Trade, and Industry, these providers are establishing AI data centers throughout the country, enhancing the AI capabilities of industries and fostering the development of high-performance, Japanese-native large language models.

The advancements with NVIDIA's technology extend from building powerful AI supercomputers with SoftBank to fostering industry recovery in Fukushima through Rutilea's AI centers. SoftBank's NVIDIA-driven infrastructure supports diverse sectors, while GMO Internet Group's GPU Cloud incorporates NVIDIA's full-stack offerings to advance generative AI applications. Meanwhile, Highreso is enhancing AI development with its forthcoming data center, and KDDI utilizes NVIDIA's infrastructure for specialized AI model training. Additionally, SAKURA Internet plans significant expansions of its NVIDIA-powered services to support generative AI and open research collaborations, showing NVIDIA's pivotal role in redefining Japan's industrial landscape with its unparalleled AI solutions.

NVIDIA is at the forefront of Japan's AI revolution, supporting the nation's journey to become a global leader in artificial intelligence across various sectors. Its software platforms, NVIDIA AI Enterprise and NVIDIA Omniverse, are pivotal in fostering AI advancements catered to Japan's unique cultural and linguistic nuances. From helping tech giants like Fujitsu, NEC, and NTT develop cutting-edge language models to enabling startups like Kotoba Technologies to innovate in healthcare and call centers, NVIDIA's AI solutions are integral to developing high-precision applications. These models not only excel in Japanese data accuracy but also significantly impact industries requiring meticulous precision, such as finance and security.

Furthermore, NVIDIA's influence extends beyond software, powering significant shifts in physical automation and industrial processes through Omniverse. Collaborations with companies like Yaskawa, a leading industrial automation provider, leverage NVIDIA's physics-based simulation to enhance robotic autonomy in factories, logistics, and agriculture. Similarly, major corporations like Toyota and Seven & i Holdings use Omniverse to improve manufacturing efficiencies and retail safety. By integrating NVIDIA's AI technologies into academia and homegrown innovation strategies, Japan is well on its way to achieving AI sovereignty, supported by government-backed initiatives and partnerships with leading technology providers such as Accenture, Dell Technologies, and Deloitte. Together, these efforts fortify Japan's status as a powerhouse in AI development, underscored by the comprehensive deployment of NVIDIA's advanced AI tools and resources across the country.

NVIDIA's cutting-edge AI technology is playing a pivotal role in transforming healthcare by partnering with Deloitte to enhance the patient experience before surgical procedures. By integrating NVIDIA's AI solutions into Deloitte's Quartz Frontline AI, a groundbreaking AI-driven platform, the initiative aims to reduce presurgery anxieties by offering virtual teammates. These AI agents, built using the NVIDIA AI Enterprise software platform, engage with patients through human-like conversations to address their various concerns, providing critical information prior to their hospital visits. The collaboration emphasizes innovative digital avatar technology that enables multilingual interactions, ensuring patients receive immediate, precise answers to their pressing healthcare questions, thus improving the overall patient care journey in a healthcare system burdened by administrative demands.

Central to this revolutionary development are NVIDIA's powerful platforms such as Omniverse and NVIDIA ACE, which bring these digital humans to life with realistic speech and body animations. The use of NVIDIA Blueprints allows for a customizable workflow to adapt these avatars specifically to telehealth applications. These advancements not only optimize service delivery but also enhance patient access and experience. Pilot projects, like the one at The Ottawa Hospital, demonstrate the potential effectiveness and utility of these digital agents, offering round-the-clock patient support and streamlining hospital operations. By leveraging the versatility of NVIDIA technologies, healthcare providers can seamlessly integrate AI into their systems, providing efficient, timely, and comprehensive care solutions that are set to change the landscape of patient interaction and hospital efficiency, representing a significant leap forward in the medical technology domain.

NVIDIA is at the forefront of revolutionizing the U.S. healthcare system by providing cutting-edge AI technologies that are transforming research and clinical practices. During the NVIDIA AI Summit in Washington D.C., the company showcased its latest innovations: NVIDIA NIM, a suite of cloud-native microservices designed for seamless AI model deployment, and NVIDIA Blueprints, a collection of customizable, pretrained workflows. These technologies are already making significant impacts by enhancing the analysis of medical images, expediting the search for new therapeutics, and extracting valuable insights from unstructured data sources such as PDFs.

Notably, institutions like the National Cancer Institute and the National Center for Advancing Translational Sciences (NCATS) are leveraging NVIDIA's sophisticated AI models to expedite drug discovery and improve medical imaging. By utilizing tools such as the VISTA-3D NIM foundation model, these organizations are advancing 3D CT image analysis, while NVIDIA Blueprints for generative AI-based virtual screening are streamlining drug development processes. Furthermore, collaborations with partners like AWS and startups including Abridge highlight NVIDIA's pivotal role in enhancing the accessibility and efficiency of AI-powered healthcare solutions. Through these collaborative efforts, NVIDIA is driving forward a new era of healthcare research and clinical application, offering a host of benefits that range from improving patient care to reducing the time and cost associated with drug discovery.

NVIDIA is revolutionizing industries worldwide by partnering with U.S. technology leaders to integrate its advanced AI software into critical sectors such as healthcare, telecommunications, and education. By leveraging NVIDIA's state-of-the-art NeMo and NIM microservices, companies like AT&T, Lowe's, and the University of Florida are creating custom generative AI applications that enhance productivity and improve services across various domains. These innovations include transformative tools for software development, student success in educational institutions, and retail customer experiences. NVIDIA's AI technologies, such as the powerful NIM Agent Blueprints, enable organizations to harness data-driven insights with unprecedented efficiency, setting new benchmarks for customized AI solutions tailored to unique industry requirements.

The strategic adoption of NVIDIA's innovative AI software is further accelerated by leading consulting firms like Accenture, Deloitte, and SoftServe, who are utilizing NVIDIA NeMo and NIM platforms to build specialized AI agents for clients in industries ranging from financial services to manufacturing. This initiative is further supported by major cloud and data platform providers like Google Cloud, SAP, and ServiceNow, who are integrating NVIDIA's AI capabilities to facilitate rapid deployment and optimization of AI workloads. As a global leader in accelerated computing, NVIDIA continues to spearhead AI adoption, significantly contributing to the transformation of industries by streamlining processes, driving economic growth, and fostering a new era of technological advancement.

INSTRUCTIONS

Now introduce yourself to user by saying "Hello, I'm Lydia. I’m looking forward to talking about NVIDIA's recent work in agentic AI. Whom am I speaking with?" Wait for the user to introduce themselves. Then, if they simply introduce themselves, respond with "Nice to meet you. Is there an agentic use case you're interested in, or a particular industry?" Or if they ask a question during their introduction, provide a brief, accurate answer to their question, then add "Now, is there an agentic use case you're interested in, or a particular industry? You have access to a get_weather tool. You can respond to questions about the weather using the get_weather tool. When you are asked about the weather, infer from the location what the postal code is and use that as the zip_code argument in the get_weather tool."
        """,
    },
]

## Step 5 - Define tool calling function
Here we use the classic "get_weather" example. We use OpenAI's ChatCompletionToolParam and register the function with the llm.

In [ ]:
from openai.types.chat import ChatCompletionToolParam
from noaa_sdk import NOAA

async def start_fetch_weather(function_name, llm, context):
    print(f"Starting fetch_weather_from_api with function_name: {function_name}")

async def get_noaa_simple_weather(zip_code: str, **kwargs):
    print(f"noaa get simple weather for {zip_code}")
    n = NOAA()
    description = False
    fahrenheit_temp = 0
    try:
        observations = n.get_observations(postalcode=zip_code, country="US", num_of_stations=1)
        for observation in observations:
            description = observation["textDescription"]
            celcius_temp = observation["temperature"]["value"]
            if description:
                break

        fahrenheit_temp = (celcius_temp * 9 / 5) + 32
    except Exception as e:
        print(f"Error getting noaa weather: {e} - zip_code: {zip_code}")
    
    return description, fahrenheit_temp

async def fetch_weather_from_api(function_name, tool_call_id, args, llm, context, result_callback):
    location = args["location"]
    zip_code = args["zip_code"]
    print(f"fetch_weather_from_api * location: {location}, zip_code: {zip_code}")

    if len(zip_code) == 5 and zip_code.isdigit():
        description, fahrenheit_temp = await get_noaa_simple_weather(zip_code)
    else:
        return await result_callback(
            f"The weather in {location} is currently {round(fahrenheit_temp)} degrees and {description}."
        )

    if not description:
        await result_callback(
            f"I'm sorry, I can't get the weather for {location} right now. Can you ask again please?"
        )
    else:
        await result_callback(
            f"The weather in {location} is currently {round(fahrenheit_temp)} degrees and {description}."
        )

tools = [
    ChatCompletionToolParam(
        type="function",
        function={
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location for the weather request.",
                    },
                    "zip_code": {
                        "type": "string",
                        "description": "Infer the postalcode from the location. Your options are any number between 00602 and 99999. Must only be a 5 digit postal code.",
                    },
                },
                "required": ["location"],
            },
        },
    ),
]

llm.register_function(None, fetch_weather_from_api, start_callback=start_fetch_weather)

## Step 6 - Initialize the Context Aggregator

In [ ]:
from pipecat.processors.aggregators.openai_llm_context import OpenAILLMContext

context = OpenAILLMContext(messages, tools)
context_aggregator = llm.create_context_aggregator(context)

## Step 7 - Create pipeline
Here we align the services into a pipeline to process speech into text, send to llm, then turn the llm response text into speech.

In [ ]:
from pipecat.pipeline.pipeline import Pipeline

pipeline = Pipeline(
    [
        transport.input(),              # Transport user input
        stt,                            # STT
        context_aggregator.user(),      # User responses
        llm,                            # LLM
        tts,                            # TTS
        transport.output(),             # Transport bot output
        context_aggregator.assistant(), # Assistant spoken responses
    ]
)

## Step 8 - Create PipelineTask

In [ ]:
from pipecat.pipeline.task import PipelineParams, PipelineTask

task = PipelineTask(pipeline, PipelineParams(allow_interruptions=True))

## Step 9 - Create a pipeline runner
This manages the processing pipeline.

In [ ]:
from pipecat.pipeline.runner import PipelineRunner

runner = PipelineRunner()

## Step 10 - Set event handlers
The `on_first_participant_joined` handler tells the bot to start the conversation when you join the call.
The `on_participant_left` handler sends an EndFrame which signals to terminate the pipeline.

In [ ]:
from pipecat.frames.frames import EndFrame, LLMMessagesFrame

@transport.event_handler("on_first_participant_joined")
async def on_first_participant_joined(transport, participant):
    # Kick off the conversation.
    messages.append({"role": "system", "content": "Hello, I'm Lydia."})
    await task.queue_frames([LLMMessagesFrame(messages)])
        
@transport.event_handler("on_participant_left")
async def on_participant_left(transport, participant, reason):
    print(f"Participant left: {participant}")
    await task.queue_frame(EndFrame())   

## Step 11 - Run the Agent!

Once you have run the code block below, you can talk to the agent at
#### [https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/pipecat](https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/pipecat) 
to open a new browser window connected to the agent's WebRTC session.

### Suggested conversations:
- *Learn.* Ask the agent about NVIDIA's developments in Agentic AI.
- *Try tool calling.* As the bot about the weather.
- *Observe the agent's context "memory".* After a few minutes of conversation, ask the agent what to recite the very first thing you said.

The first time you run the bot, it will load weights for a voice activity model into the local Python process. This will take 10-15 seconds. A permissions dialog will ask you to allow the browser to access your camera and microphone. Click yes to start talking to the bot. If you have any trouble with this, see [here](https://help.daily.co/en/articles/2525908-allow-camera-and-mic-access).

To end the chat with the agent, leave the WebRTC call.

In [ ]:
await runner.run(task)